In [1]:
import pandas as pd
from collections import Counter
from PIL import Image,ImageDraw,ImageFont
from scipy.stats import spearmanr
import os
import numpy as np

In [2]:
limefeat_alphasort = pd.read_csv("limefeat_alphasort.csv")
officer_table = pd.read_csv("officer_table.csv")
df = pd.read_csv("lime-robustness-all-score-outcome.csv")

In [3]:
officer_table.sort_values(by="unit_score",ascending=False,inplace=True)
officer_table.reset_index(drop=True,inplace=True)

In [4]:
df.sort_values(by=['unit_score','officer_id','nft'],ascending=[False,True,True],inplace=True)
df.reset_index(drop=True,inplace=True)

In [5]:
def relister(liststring):
    tmp = liststring.lstrip("[").rstrip("]")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [6]:
def setstring2list(setstring):
    tmp = setstring.lstrip("{").rstrip("}")
    tmp = tmp.split(",")
    tmp = [item.translate(None," '") for item in tmp]
    return tmp

In [7]:
officer_table['feats'] = officer_table.feats.apply(relister)
df['unq'] = df.unq.apply(setstring2list)

#### Feature categories

In [8]:
cathue = {
    'ts':'0',
    'fi':'32',
    'ir':'60',
    'shifts':'115',
    'dispatch':'180',
    'arrests':'240',
    'ic':'270',
    'demarrests':'300',
    'ocnd':'200'
    }

catsat = {
    'ts':'100%',
    'fi':'100%',
    'ir':'100%',
    'shifts':'100%',
    'dispatch':'100%',
    'arrests':'100%',
    'ic':'100%',
    'demarrests':'100%',
    'ocnd':'100%'
    }

catval = {
    '1d':'5%',
    '1w':'25%',
    '1m':'50%',
    '1y':'75%',
    '5y':'100%',
    'all':'100%',
    }

In [9]:
impath = 'offtableart.png'
rectheight = 10
rectwidth = 10

#### Row Labels

In [10]:
im = Image.open(impath)
w = im.width
h = im.height
x = 100
canvas = Image.new('RGB',((w+x),(h)),'hsl(0,0%,100%)')
canvas.paste(im,(x,0))
im = None

/Library/Python/2.7/site-packages/PIL/Image.py:2261: DecompressionBombWarning: Image size (404434800 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [11]:
draw = ImageDraw.Draw(canvas)
for i in df.index:
    officer_id = df.officer_id.loc[i]
    nft = df.nft.loc[i]
    nsmp = df.nsmp.loc[i]
    
    text = str(officer_id)+","+str(nft)+","+str(nsmp)
    font = ImageFont.truetype('VeraMono.ttf', rectheight)
    
    xpos = 5
    ypos = i * rectheight
    draw.text((xpos,ypos),text=text,font=font,fill="black")

In [12]:
#canvas.save('/Users/damoncrockett/Desktop/tmp.png')

#### Column Labels

In [13]:
w = canvas.width
h = canvas.height
y = 400
unicanvas = Image.new('RGB',((w),(h+y)),'hsl(0,0%,100%)')
unicanvas.paste(canvas,(0,y))
canvas = None

In [14]:
draw = ImageDraw.Draw(unicanvas)
for i in limefeat_alphasort.index:
    feature = limefeat_alphasort.featstr.loc[i]
    text = str(feature)
    
    font = ImageFont.truetype('VeraMono.ttf', rectwidth)
    fontWidth, fontHeight = font.getsize(text)
    
    imtmp = Image.new('RGBA', (fontWidth, fontHeight))
    drawtmp = ImageDraw.Draw(imtmp)
    drawtmp.text((0,0),text=text,font=font,fill="black")
    imtmp = imtmp.rotate(90,expand=1)
    
    xpos = i * rectwidth + x - 2
    ypos = y - imtmp.height - 2
    unicanvas.paste(imtmp,(xpos,ypos),imtmp)
    
    linex = i * rectwidth + x
    liney = y
    draw.line([(linex,liney),(linex,unicanvas.height)],width=1,fill='black')

In [15]:
unicanvas.save('/Users/damoncrockett/Desktop/tmp.png')